In [1]:
import sys
sys.path.append(r"D:\Scripts1\Code\ActPEA\CODE\Actu_Colums.py")
import pandas as pd
import matplotlib as mptl
import Actu_Colums as Ac
from openpyxl import load_workbook
from typing import Optional
import glob,os,re
import numpy as np
from collections import defaultdict
from babel.dates import format_date, format_datetime, Locale
import datetime
import locale
locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')
import pickle

# La estrucutra es: 
- https://miro.com/welcomeonboard/MXJUUHNkTk5LWFFtamRMUDBUTnAxNVhDVjJyeTlzYnN5SzhzQXN6MkhwNzBsQXZoSzNBcHNsSlFMT1lIdlRrZXwzNDU4NzY0NTkxMzU2NDUxNTY0fDI=?share_link_id=885670918178


# Funciones 

In [50]:
Today_str = datetime.date.today().strftime('%d-%m-%Y')
Today_D_M = Today_str[0:5]
Today_D_M = [Today_D_M[0:2],'.',Today_D_M[3:6]]
Today_D_M = ''.join(Today_D_M)

def addSite(EA_act_df : pd.DataFrame, SAP_4_use : pd.DataFrame):
    """Funcione que toma las tablas SAP y extrae los sites y los codigos de AHI
        Devuelve un DF con la Informacion Añadida ahi"""
    df_merged = pd.merge(EA_act_df,SAP_4_use,on='CONCATENADO',how='left')
    df_merged = df_merged.drop_duplicates(subset='CONCATENADO')
    df_merged['PEP Desc'] = df_merged['PEP Desc'].astype(str)
    df_merged['ID_SITE_SAP'], df_merged['SITE_SAP'] = zip(*df_merged['PEP Desc'].apply(lambda x: re.split('(?<=\d)(?=[a-zA-Z])', x) 
                                                                                           if re.search('(?<=\d)(?=[a-zA-Z])', x) else [np.nan,x]))

    # Verifica si las columnas 'SITE' e 'ID_SITIO' existen, si no, las crea
    if 'SITE' not in df_merged.columns:
        df_merged['SITE'] = np.nan
    if 'ID_SITIO' not in df_merged.columns:
        df_merged['ID_SITIO'] = np.nan

    # Reemplaza los valores NaN en 'SITE' e 'ID_SITIO' con los valores de 'SITE_SAP' e 'ID_SITE_SAP'
    df_merged['SITE'] = df_merged['SITE'].combine_first(df_merged['SITE_SAP'])
    df_merged['ID_SITIO'] = df_merged['ID_SITIO'].combine_first(df_merged['ID_SITE_SAP'])

    df_merged.SITE = df_merged.SITE.str.strip() # Quito espacios

    EA_act_df = df_merged.drop(columns=['ID_SITE_SAP','SITE_SAP','Fecha OC']) # Elimino las columnas que use para el Merge 
    return EA_act_df
    
def get_recent_df_B(Carpeta_path: str, sheet_name: str):
    """ Devuelve el df de la hoja especifica, del archivo mas reciente de la carpeta especificada"""
    Path_n= Carpeta_path + '/*'
    tipo_de_archivo = '*.xlsx'
    # Busca el archivo más reciente
    archivos = glob.glob(Path_n + tipo_de_archivo)
    # Filtra los archivos que no contienen "_" en su nombre
    archivos_sin_guion_bajo = [archivo for archivo in archivos if "_" not in os.path.basename(archivo)]
    archivo_mas_reciente = max(archivos_sin_guion_bajo, key=os.path.getctime)
    nombre_del_archivo_N = os.path.basename(archivo_mas_reciente)
    print(archivo_mas_reciente)
    # Lee el archivo sin especificar el tipo de datos
    df = pd.read_excel(archivo_mas_reciente , sheet_name=sheet_name)
    # Si la columna "COMENTARIO" existe, cambia su tipo de datos a str
    if 'COMENTARIO' in df.columns:
        df['COMENTARIO'] = df['COMENTARIO'].astype(str)
    return df,nombre_del_archivo_N



def get_recent_df_by_N(Carpeta_path: str, sheet_name: str, prefijo_nombre_archivo: str):
    """ Devuelve el df de la hoja especifica, del archivo mas reciente de la carpeta especificada"""
    # Busca el archivo más reciente que comienza con el prefijo_nombre_archivo
    archivos = glob.glob(os.path.join(Carpeta_path, prefijo_nombre_archivo + '*.xlsx'))
    archivo_mas_reciente = max(archivos, key=os.path.getctime)
    nombre_del_archivo_N = os.path.basename(archivo_mas_reciente)
    print(archivo_mas_reciente)
    # Lee el archivo sin especificar el tipo de datos
    df = pd.read_excel(archivo_mas_reciente , sheet_name=sheet_name)
    # Si la columna "COMENTARIO" existe, cambia su tipo de datos a str
    if 'COMENTARIO' in df.columns:
        df['COMENTARIO'] = df['COMENTARIO'].astype(str)
    return df, nombre_del_archivo_N
def get_recent_df(Carpeta_path: str, sheet_name: str):
    """ Devuelve el df de la hoja especifica, del archivo mas reciente de la carpeta especificada"""
    Path_n= Carpeta_path + '/*'
    tipo_de_archivo = '*.xlsx'
    # Busca el archivo más reciente
    archivos = glob.glob(Path_n + tipo_de_archivo)
    archivo_mas_reciente = max(archivos, key=os.path.getmtime)
    nombre_del_archivo_N = os.path.basename(archivo_mas_reciente)
    print(archivo_mas_reciente)
    # Lee el archivo sin especificar el tipo de datos
    df = pd.read_excel(archivo_mas_reciente , sheet_name=sheet_name)
    # Si la columna "COMENTARIO" existe, cambia su tipo de datos a str
    if 'COMENTARIO' in df.columns:
        df['COMENTARIO'] = df['COMENTARIO'].astype(str)
    return df,nombre_del_archivo_N

def get_recent_csv(Carpeta_path: str):
    """ Devuelve el df del archivo csv más reciente de la carpeta especificada"""
    Path_n = Carpeta_path + '/*'
    tipo_de_archivo = '*.csv'
    # Busca el archivo más reciente
    archivos = glob.glob(Path_n + tipo_de_archivo)
    archivo_mas_reciente = max(archivos, key=os.path.getmtime)
    nombre_del_archivo_N = os.path.basename(archivo_mas_reciente)
    print(archivo_mas_reciente)
    # Lee el archivo sin especificar el tipo de datos
    df = pd.read_csv(archivo_mas_reciente)
    # Si la columna "COMENTARIO" existe, cambia su tipo de datos a str
    if 'COMENTARIO' in df.columns:
        df['COMENTARIO'] = df['COMENTARIO'].astype(str)
    return df, nombre_del_archivo_N

def Process_PAP(PAP:pd.DataFrame):
    PAP_f = PAP[['OC Posición','N° Sol','Estado','Acción','Nombre responsable','Id.SIte',
                 'SIte','# Días','F.Creación','F.Modifica']]
    ##Filtro solo del norte
    PAP_f = PAP_f.dropna(subset=['Id.SIte'])
    PAP_f = PAP_f[PAP_f['# Días'] < 500]
    
    PAP_f_N = PAP_f[PAP_f['Id.SIte'].str.startswith(('L','T','SAD','CL','CAC'))].copy()
    PAP_f_N = PAP_f_N.rename(columns={'N° Sol': 'PAP', #Rename
                                   'SIte': 'SITE',
                                   'Estado': 'ESTADO_PAP',
                                    '# Días': 'ANTIGUAMIENTO_PAP',
                                    'Id.SIte': 'ID Site',
                                 'Nombre responsable': 'RESPONSABLE_PAP' })
    PAP_S = Ac.split_ocs(PAP_f_N) # Spliteo OCs
    return PAP_S
def pre_proces(df: pd.DataFrame ,columns_2str: list[str] ,column_filter: str,C_format) -> pd.DataFrame: 
    """Funcion que filtra y convierte a str ciertas columnas especificadas """
    ## Preprosecing of PREP_NEW
    df = convert_columns(df.copy(),columns_2str,C_format ) #Convert to str a key column
    df_EI = df.loc[df[column_filter] == 'Eduardo Iberico']#Filter
    df_EI = df_EI.copy()  # Crea una copia del DataFrame original para evitar modificar los datos originales
    return df_EI
def convert_columns(df, columns,type):
    for column in columns:
        df[column] = df[column].astype(type)
    return df

def split_column_melt(df, column, delimiter):
    # Dividimos la columna en dos partes
    df_copy=df.copy()
    df_copy[[f'{column}1', f'{column}2']] = df_copy[column].str.split(delimiter, expand=True)

    # Eliminamos la columna original
    df_copy = df_copy.drop(columns=[column])

    # Usamos melt para "derretir" las columnas en una sola columna
    df1 = df_copy.melt(id_vars=[col for col in df_copy.columns if col != f'{column}1' and col != f'{column}2'], 
                  value_vars=[f'{column}1', f'{column}2'], 
                  value_name=column)

    # Eliminamos la columna 'variable'
    #df1 = df1.drop(columns=['0'])
    return df1
def tratar_codigo(codigo):
    # Si el código tiene una letra extra en el medio
    if not re.match(r'^[A-Za-z]{2}\d{4}$', codigo):
        # Quitar la tercera letra si es una letra
        if codigo[2].isalpha():
            codigo = codigo[:2] + codigo[3:]
        # Quitar los números extras al final si son números
        while len(codigo) > 6 and codigo[-1].isdigit():
            codigo = codigo[:-1]
    return codigo
    
map_Contr = {'HUAWEI':'HUAWEI DEL PERU S.A.C.',
       'UNION ELECTRICA' : 'UNION ELECTRICA SA SUCURSAL DEL PER',
       'HERASI' : 'HERASI S.A.C',
       'TELRAD' : 'TELRAD PERU S.A.',
       'CICSA'  : 'CICSA PERU S.A.C.',
         'LPS'   :'LICENCIAS PROYECTOS Y SEGURIDAD Y S',
            'TELRAD' : 'TELRAD PERU S.A.',
            'SIAE': 'SIAE MICROELETTRONICA S.A.C',
            'REDSETEL':'RED DE SERVICIOS Y TELECOMUNICACION ES PERU SAC REDSETEL PERU S.A.C.',
            'SEMI':'SEMI PERU MONTAJES INDUSTRIALES S.A.C.',
            'MEG':'MEG PERU S.A.C.',
            'COMFUTURA':'COMUNICACION FUTURA SOCIEDAD ANONIM',
            'CJ TELECOM':'CJ TELECOM S.A.C'}

def compact_rows(df:pd.DataFrame, columns:list, delimiter:str ='/'):
    for col in OT_cut_RRL_2M.columns:
        df[col] = df[col].astype(str)
    
    # Usar 'join' como función de agregación para concatenar los valores
    agg_func = {col: lambda x: '/'.join(x.unique()) for col in df.columns if col not in  columns}
    
    # Agrupar por 'OC Posición' y aplicar la función de agregación
    df1 = df.groupby(columns).agg(agg_func).reset_index()
    return df1
def contar_prefijos(lista):
        contador = defaultdict(int)
        for cadena in lista:
            for i in range(1, len(cadena) + 1):
                prefijo = cadena[:i]
                contador[prefijo] += 1
        return contador

def normalize_company_names(df, column):
    """Normaliza los nombres de las empresas en la columna especificada del DataFrame."""
    # Reemplaza "SAC" o "S. A. C." al final de los nombres de las empresas con "S.A.C"
    df[column] = df[column].str.replace(r"(SAC|S\. ?A\. ?C\.)$", "S.A.C.", regex=True)
    return df


def merge_dataframes(df1, df2, merge_on, index):
    merged_df = pd.merge(df1, df2, on=merge_on, how='left')
    merged_df = merged_df.drop_duplicates()
    #print("filas originales: ",len(merged_df))
    #print("filas index: ",len(index))
    merged_df.set_index(index, inplace=True)
    return merged_df
def update_and_rename(df1, update_cols, new_names):
    df = df1.copy()
    for col in update_cols:
        df[col[0]].update(df[col[1]])
    df.rename(columns=new_names, inplace=True)
    df.drop(columns=[col for sublist in update_cols for col in sublist[1:]], inplace=True)
    return df
def combine_and_rename(df1, combine_cols, new_names):
    df = df1.copy()
    for cols in combine_cols:
        df[cols[0]] = df[cols[0]].combine_first(df[cols[1]])
    df.rename(columns=new_names, inplace=True)
    df.drop(columns=[col for sublist in combine_cols for col in sublist[1:]], inplace=True)
    return df
def load_merge(directorio:str):
    # Dicc para almacenar los DataFrames
    dfs = {}
    # Itera sobre todos los archivos en el directorio
    for filename in os.listdir(directorio):
        file_path = os.path.join(directorio, filename)
        # Verifica si el path es un archivo y no un directorio
        if os.path.isfile(file_path):
            df = pd.read_excel(file_path)  
            # Obtiene el nombre del archivo sin la extensión
            nombre_sin_extension = os.path.splitext(filename)[0]
            # Añade el DataFrame al diccionario
            dfs[nombre_sin_extension] = df
    ## Etiqueto cada Df por su temporalidad, añado una columna mas con dicha etiqueta         
    for tiempo, df in dfs.items():
        df['TIME'] = tiempo
    # Combina todos los dataframes en uno solo
    df_combinado = pd.concat(dfs.values())
    return df_combinado    
def process_to_bcsv(df : pd.DataFrame,ruta_del_csv : str,Fecha: str):

    df['TIME'] = Fecha
    pivot_table = pd.pivot_table(df, values='EN_PROC_USD', index=['TIME','FECHA_DOC','RESPONSABLE_DE_EA','Estado de EA'], aggfunc=pd.Series.sum) #Agrupo 
    df_reset = pivot_table.reset_index(drop=False)
    #convetir la columna del agrupado al formato d efehca 
    # Convierte la columna 'Fecha' a datetime
    df_reset['TIME'] = pd.to_datetime(df_reset['TIME'],format = '%d-%m-%Y')
    
    # Formatea la columna 'Fecha'
    df_reset['TIME Format'] = df_reset['TIME'].apply(lambda x: format_date(x, 'EEE dd-MM-yyyy', locale=Locale('es', 'ES')))
    #return df_reset
    df_reset.to_csv(ruta_del_csv, mode='a', header=False,index=False)
def get_OCS (df:pd.DataFrame): ## Correguir esta funcion(Solo debe hacer una cosa)
    EA_PAP_clean_4_SAP = df[(df.SITE.isna())|(df.SITE == 'nan')] #filto2
    listocs = EA_PAP_clean_4_SAP['CONCATENADO'].astype(str).tolist() # Creo lsitas de OCS
    resultado = [valor[4:] for valor in listocs] # Tomo los valores luego del 4500
    
    a,b= MostCL_prefix(resultado)
    return a,b
def MostCL_prefix(lista : list):
    b = 0
    list_n = []
    list_n1 = {}
    contador = contar_prefijos(lista)
    for key in contador.keys(): 
       ###Aqui iria la nueva condicional### 
        if contador[key] < b:
               list_n.append(key)
        b = contador[key]
    claves = list(contador.keys())
    for clave_dada in list_n:
           indice = claves.index(clave_dada)
           clave_anterior = claves[indice - 1]
           #list_n.append(clave_anterior)
           list_n1[clave_anterior] = contador[clave_anterior]
    return list_n1,contador
def limpiar_id(df, col_id, col_nombre):
    # Convertir la columna de ID a string para poder hacer la comparación
    df[col_id] = df[col_id].astype(str)
    
    # Crear una función para limpiar el nombre
    def limpiar_nombre(row):
        nombre = str(row[col_nombre])  # Convertir el nombre a string
        id_actual = row[col_id]
        
        # Dividir el nombre por "_" y "-"
        partes = re.split('_|-| ', nombre)
        
        # Si el ID está en el nombre, eliminarlo
        if id_actual in partes:
            partes.remove(id_actual)
        
        # Devolver el nombre limpio
        return ' '.join(partes)
    
    # Aplicar la función de limpieza a cada fila del DataFrame
    df[col_nombre] = df.apply(limpiar_nombre, axis=1)
    
    return df


# Cargo Archivos 

In [3]:
PEXT,_ = get_recent_df(r'D:\Scripts1\Code\ActPEA\archvis\PEXT_S',sheet_name='General')

D:\Scripts1\Code\ActPEA\archvis\PEXT_S\SEGUIMIENTO DE PROYECTOS 2024-3.xlsx


C:\Users\C26764\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [4]:
PEA,nombre_del_archivo_N = get_recent_df_by_N(r'C:\Users\C26764\America Movil Peru S.A.C\EAS - 2',sheet_name= 'DATA',
                                              prefijo_nombre_archivo='Pendiente de entrada al') 

C:\Users\C26764\America Movil Peru S.A.C\EAS - 2\Pendiente de entrada al 30.06.24.xlsx


In [5]:
from datetime import datetime ## Calculo la aniguedad del Reporte 
Fecha_reporte = nombre_del_archivo_N[-13:-8].replace('.','-')
Fecha_reporte_DT = datetime.strptime(Fecha_reporte, "%d-%m")
Today2C = datetime.strptime(Today_str[:-5], "%d-%m")
a = Today2C - Fecha_reporte_DT
if Fecha_reporte_DT == Today2C:
    reporte_old = False
    print("El reporte es actual")
elif Fecha_reporte_DT < Today2C:
    reporte_old = True
    print(f"El reporte es de hace {a.days} dias")


El reporte es de hace 4 dias


In [6]:
##########PAP################ MAIN_2
PAP_O,_ = get_recent_df_B(r'D:\Scripts1\Code\ActPEA\archvis\PAP','Hoja2')# PAP de Otros
PAP_A,_ = get_recent_df(r'D:\Scripts1\Code\ActPEA\archvis\PAP\Administrativo','Hoja2')# PAP de Admin
PAP = pd.concat([PAP_O,PAP_A])
PAP_N_S = Process_PAP(PAP)

D:\Scripts1\Code\ActPEA\archvis\PAP\PAP 04.07.xlsx
D:\Scripts1\Code\ActPEA\archvis\PAP\Administrativo\PAP_A_04.07.2024.xlsx


In [7]:
################## Archivo PAP OTs #################################
OT,_= get_recent_df(r'D:\Scripts1\Code\ActPEA\archvis\OTs','Hoja2')
OT = OT[OT['Status OT'] != 'PDTE CONTRATA'] #	
OT = OT[OT['Status OT'] != 'PDTE RESPONSABLE'].copy()
def cortar_hora(fecha_hora_str):
    return fecha_hora_str.split(" ")[0]

# Aplica la función a toda la columna 'fecha_hora'
OT['Fecha de Creación'] = OT['Fecha de Creación'].apply(cortar_hora)
OT['Fecha de Creación'] = pd.to_datetime(OT['Fecha de Creación'], format='%d/%m/%Y',dayfirst=True)
fecha_limite = pd.Timestamp('2022-01-01')
OT_cut = OT[OT['Fecha de Creación'] > fecha_limite].copy() # Corto las OTs solo de este año 
OT_cut = normalize_company_names(OT_cut,'Contrata')


D:\Scripts1\Code\ActPEA\archvis\OTs\OTS 01.07.2024.xlsx


# Preproces 


## OTs-RollOut

In [8]:
Proyectos = ['ROLLOUT - 2023','ROLLOUT - 2022','ROLLOUT - 2024']
OT_cut_RRL = OT_cut[OT_cut.Etiqueta.isin(Proyectos)].copy()
OT_cut_RRL.rename(columns={'Codigo de Site':'ID_SITIO',
                           'Contrata':'PROVEEDOR',
                           'Nombre de Site':'SITE'},inplace=True)
OT_cut_RRL_2M = OT_cut_RRL[['OT','ID_SITIO','SITE','Proyecto','PROVEEDOR','Status OT']].copy()
## Agg 4 singles index
OT_agg_ID_PRO = compact_rows(OT_cut_RRL_2M,['ID_SITIO','PROVEEDOR'],'/')
OT_agg_ID_NM_PRO = compact_rows(OT_cut_RRL_2M,['ID_SITIO','SITE','PROVEEDOR'],'/')


## OTs-Moder

In [9]:
OT_cut_M = OT_cut[OT_cut.Proyecto == 'EXPANSIÓN'].copy()

OT_cut_M.rename(columns={'Codigo de Site':'ID_SITIO',
                           'Contrata':'PROVEEDOR',
                           'Nombre de Site':'SITE'},inplace=True)
OT_cut_M = OT_cut_M[['OT','ID_SITIO','SITE','Proyecto','PROVEEDOR','Status OT']].copy()

OT_cut_M_ID_PRO = compact_rows(OT_cut_M,['ID_SITIO','PROVEEDOR'])
OT_cut_M_ID_N_PRO = compact_rows(OT_cut_M,['ID_SITIO','SITE','PROVEEDOR'])

OT_cut_M_ID_PRO.rename(columns={'Proyecto':'RESPONSABLE_DE_EA'},inplace=True)
OT_cut_M_ID_PRO['RESPONSABLE_DE_EA'] = 'ANGGIE'

OT_cut_M_ID_N_PRO.rename(columns={'Proyecto':'RESPONSABLE_DE_EA'},inplace=True)
OT_cut_M_ID_N_PRO['RESPONSABLE_DE_EA'] = 'ANGGIE'

## OTs-Energia

In [10]:
fecha_limite = pd.Timestamp('2023-02-15')
OT_cut = OT[OT['Fecha de Creación'] > fecha_limite].copy() # Corto las OTs solo de este año 
OT_cut = normalize_company_names(OT_cut,'Contrata')

OT_cut_E = OT_cut[OT_cut['Tipo Req'] == 'MANT. MEJORA DE RED'].copy()
Proy_energia = ['AC ESTABILIZADA','AA','INCREMENTO DE POTENCIA (INTERNO - CONSTRUCCIÓN)',
                'AC COMERCIAL','ENERGÍA DC','AMPLIACION DE POTENCIA']
OT_cut_E.rename(columns={'Codigo de Site':'ID_SITIO',
                           'Contrata':'PROVEEDOR',
                           'Nombre de Site':'SITE'},inplace=True)

OT_cut_E = OT_cut_E[OT_cut_E.Proyecto.isin(Proy_energia)][['OT','ID_SITIO','SITE','Proyecto','PROVEEDOR','Status OT']].copy()
OT_cut_E_ID_PRO = compact_rows(OT_cut_E,['ID_SITIO','PROVEEDOR'])
OT_cut_E_ID_PRO.rename(columns={'Proyecto':'RESPONSABLE_DE_EA'},inplace=True)
OT_cut_E_ID_PRO['RESPONSABLE_DE_EA'] = 'JORGE'

OT_cut_E_ID_PRO['Status OT'].value_counts()

Status OT
ING               207
CW/TI              45
FINALIZADO         17
FINALIZADO/ING      1
ING/CW/TI           1
Name: count, dtype: int64

## PEA 

In [11]:
############################################### PEA #####################################################################
PEA_EI_NC = pre_proces(PEA,['DOC_PREC', 'POS_PREC','CONCATENADO'],'SOLICITANTE','int64') # Truca el . decimal
PEA_EI_NC = normalize_company_names(PEA_EI_NC,'PROVEEDOR')
PEA_EI = pre_proces(PEA_EI_NC,['DOC_PREC', 'POS_PREC','CONCATENADO'],'SOLICITANTE','str')  # 2 STR 
#####################################################################################################
###############################################################################################3
PEA_EI.loc[:, "OC Posición"] = PEA_EI.loc[:, "DOC_PREC"].str.cat(PEA_EI.loc[:, "POS_PREC"], sep= ":")
Cash_In = PEA_EI["EN_PROC_USD"].sum() #Mont of USD 
PEA_EI_NCD = PEA_EI[PEA_EI.ID_SITIO.isna()]
PEA_EI_CD = PEA_EI[~PEA_EI.ID_SITIO.isna()].copy()
# Tratar los códigos
PEA_EI_CD['ID_SITIO'] = PEA_EI_CD['ID_SITIO'].apply(tratar_codigo)
PEA_EI = pd.concat([PEA_EI_CD,PEA_EI_NCD])

## PEXT

In [12]:
PEXT.loc[:,'REQ'] = PEXT.loc[:,'REQ'].fillna(0000)

In [13]:

PEXT_N = PEXT[PEXT.REGION == 'NORTE'].iloc[:,1:][['REQ','ID','SITE','RESPONSABLE DE IMPLEMENTACION','CONTRATISTA','ESTATUS GENERAL','ORDEN DE COMPRA','POS']]
PEXT_N['REQ'] = PEXT_N['REQ'].astype('int32')
PEXT_N[['ORDEN DE COMPRA', 'POS']] = PEXT_N[['ORDEN DE COMPRA', 'POS']].astype(str)
PEXT_N.loc[:, "OC Posición"] = PEXT_N.loc[:, "ORDEN DE COMPRA"].str.cat(PEXT_N.loc[:, "POS"], sep= ":")
PEXT_N.replace({'ORDEN DE COMPRA': {'nan': np.nan}, 'POS': {'nan': np.nan}, 'OC Posición': {'nan:nan': np.nan}}, inplace=True)
PEXT_N.drop(columns=['ORDEN DE COMPRA','POS'], inplace=True)
PEXT_N_OCs = PEXT_N[~PEXT_N['OC Posición'].isna()].copy()
PEXT_N_OCs['REQ'] = PEXT_N_OCs['REQ'].astype(str)


In [14]:
for col in PEXT_N_OCs.columns:
    PEXT_N_OCs[col] = PEXT_N_OCs[col].astype(str)

agg_func = {col: 'first' for col in PEXT_N_OCs.columns if col != 'OC Posición'}
# Agrupar por 'Asignatura' y aplicar la función de agregación
PEXT_agg = PEXT_N_OCs.groupby('OC Posición').agg(agg_func).reset_index()


# PROCES MAIN
 - @TODO, Cambiar el orden del PAP y la tabla SAP 

## Actualizo alguna informacion de la tablas SAP(SAP_4_use/DB PAP) -- Solo se ejecuta cuando hay nuevo reporte 

### TABLAS SAP - DB SITE PAP

In [15]:
PEA_EI.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1297 entries, 49 to 10898
Data columns (total 49 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   CONCATENADO               1297 non-null   object        
 1   DOC_PREC                  1297 non-null   object        
 2   POS_PREC                  1297 non-null   object        
 3   FECHA_DOC                 1297 non-null   datetime64[ns]
 4   Estrategia de Liberación  1294 non-null   object        
 5   Cod.Solicitante           1297 non-null   object        
 6   SOLICITANTE               1297 non-null   object        
 7   TIPO_DOC                  1297 non-null   object        
 8   TIPO DE MATERIAL          1297 non-null   object        
 9   MATERIAL                  1297 non-null   int64         
 10  TEXTO                     1297 non-null   object        
 11  COD_PROVEEDOR             1294 non-null   float64       
 12  PROVEEDOR              

In [16]:
if not reporte_old:
    directorio = r'D:\Scripts1\Code\ActPEA\archvis\Tablas SAP'
    SAP_4_use = (load_merge(directorio)
                  .pipe(Ac.convert_columns_to_str, ['OC','Pos']) # Uso pipe para poder encadenar funciones, dentro de pipe puedo pasar cualquier funcion a ejecutar en el df con sus argumentos
                  .assign(CONCATENADO=lambda df: df["OC"].str.cat(df["Pos"], sep=""))
                  .loc[:, ['CONCATENADO','PEP Desc','Fecha OC']]
                  .replace({'PEP Desc': {'TJ5125-SANTIAGO DE CHUCO' : 'TJ5125-SANTIAGO_DE_CHUCO'}})
                  .dropna(subset=['PEP Desc']))

    EA_4 = PEA_EI[PEA_EI.CONCATENADO.astype(str).str.startswith('4')]
    EA_2 = PEA_EI[~PEA_EI.CONCATENADO.astype(str).str.startswith('4')]

    EA_SAP = pd.concat([addSite(EA_4.astype(str), SAP_4_use).drop(columns=['PEP Desc']), EA_2], axis=0)
    EA_SAP.loc[:,'SITE'].replace('nan',np.nan,inplace=True)
    EA_SAP_SITE = pd.merge(EA_SAP,PAP_DB,on='ID_SITIO',how='left')
    EA_SAP_SITE = combine_and_rename(EA_SAP_SITE,[('SITE_x', 'SITE_y')],{'SITE_x':'SITE'})
    a,b = get_OCS(EA_SAP_SITE)
    EA_SAP_SITE.to_csv(r'D:\Scripts1\Code\ActPEA\CODE\Temps\Base_EAS',index=False)
    print("Aun faltan descargar: ver variable 'a'&'b'")
    print("Actualizo info de las Tablas SAP")
else:
    print("El reporte es OLD, uso la base")


El reporte es OLD, uso la base


In [17]:
a

datetime.timedelta(days=4)

## Actualizo alguna informacion del PAP(PAP2M)

In [18]:
EA_SAP_load = pd.read_csv(r'D:\Scripts1\Code\ActPEA\CODE\Temps\Base_EAS') # literalmete uso la base 
#EA_SAP_load.drop(columns=['PEP Desc'],inplace=True)

In [19]:
EA_SAP_load.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1297 entries, 0 to 1296
Data columns (total 50 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   CONCATENADO               1297 non-null   int64  
 1   DOC_PREC                  1297 non-null   int64  
 2   POS_PREC                  1297 non-null   int64  
 3   FECHA_DOC                 1297 non-null   object 
 4   Estrategia de Liberación  1294 non-null   object 
 5   Cod.Solicitante           1297 non-null   object 
 6   SOLICITANTE               1297 non-null   object 
 7   TIPO_DOC                  1297 non-null   object 
 8   TIPO DE MATERIAL          1297 non-null   object 
 9   MATERIAL                  1297 non-null   int64  
 10  TEXTO                     1297 non-null   object 
 11  COD_PROVEEDOR             1294 non-null   float64
 12  PROVEEDOR                 1294 non-null   object 
 13  CENTRO_GESTOR             1297 non-null   object 
 14  POS_PRES

In [20]:
len(EA_SAP_load)

1297

In [21]:
PAP2M = PAP_N_S[['PAP','ESTADO_PAP','SITE','ANTIGUAMIENTO_PAP','OC Posición','F.Creación']]   

EA_SAP_PAP = pd.merge(EA_SAP_load,PAP2M,how='left',on='OC Posición')
EA_SAP_PAP = update_and_rename(EA_SAP_PAP,[('SITE_x', 'SITE_y')],{'SITE_x':'SITE'})


### Clean Duplicated Rows
EA_SAP_PAP.rename(columns={'DOC_PREC':'NUMERO OC'},inplace=True) 
EA_SAP_PAP['F.Creación'] = pd.to_datetime(EA_SAP_PAP['F.Creación'], errors='coerce')
EA_SAP_PAP['F.Creación'] = pd.to_datetime(EA_SAP_PAP['F.Creación'],format="%d/%m/%Y %I:%M:%S %p")

EA_duplicated = EA_SAP_PAP[EA_SAP_PAP.duplicated(subset='CONCATENADO',keep=False)].copy()
EA_ND = EA_SAP_PAP[~EA_SAP_PAP.duplicated(subset='CONCATENADO',keep=False)].copy()

EA_duplicated.sort_values('F.Creación',ascending=False, inplace=True)
EA_duplicated_NR = EA_duplicated[EA_duplicated.ESTADO_PAP != 'Rechazado'].sort_index(ascending=False)

EA_duplicated_ND = EA_duplicated_NR.drop_duplicates(subset='CONCATENADO',keep='first')
EA_SAP_PAP = pd.concat([EA_ND,EA_duplicated_ND])

In [22]:
EA_SAP_PAP.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1297 entries, 0 to 124
Data columns (total 54 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   CONCATENADO               1297 non-null   int64         
 1   NUMERO OC                 1297 non-null   int64         
 2   POS_PREC                  1297 non-null   int64         
 3   FECHA_DOC                 1297 non-null   object        
 4   Estrategia de Liberación  1294 non-null   object        
 5   Cod.Solicitante           1297 non-null   object        
 6   SOLICITANTE               1297 non-null   object        
 7   TIPO_DOC                  1297 non-null   object        
 8   TIPO DE MATERIAL          1297 non-null   object        
 9   MATERIAL                  1297 non-null   int64         
 10  TEXTO                     1297 non-null   object        
 11  COD_PROVEEDOR             1294 non-null   float64       
 12  PROVEEDOR                 

## Actualizo Name Sites y codes de las Tablas SAP 

## Añado info de PEXT 
- Como es por OC, no me conviene separa los vacios

In [23]:
EA_PAP_SAP_PEXT = pd.merge(EA_SAP_PAP,PEXT_agg,on='OC Posición',how='left')
EA_PAP_SAP_PEXT = combine_and_rename(EA_PAP_SAP_PEXT,[('SITE_x', 'SITE_y')],{'SITE_x':'SITE'})

## OTS

### Merge a 3 columnas 

In [24]:
EA_PAP_PEXT_OTS_3 = pd.merge(EA_PAP_SAP_PEXT,OT_agg_ID_NM_PRO,on=['ID_SITIO','SITE','PROVEEDOR'],how='left')
EA_PAP_PEXT_OTS_3['ESTATUS GENERAL'] = EA_PAP_PEXT_OTS_3['ESTATUS GENERAL'].combine_first(EA_PAP_PEXT_OTS_3['Status OT'])
EA_PAP_PEXT_OTS_3.drop(columns=['Status OT'],inplace=True) 

##########Separo por los que tienen OT / al reciduo le hago el doble merge y luego junto ambos 
EA_PAP_PEXT_OTS_3_R =EA_PAP_PEXT_OTS_3[~EA_PAP_PEXT_OTS_3.OT.isna()]
EA_PAP_PEXT_OTS_3_NR = EA_PAP_PEXT_OTS_3[EA_PAP_PEXT_OTS_3.OT.isna()].copy()
EA_PAP_PEXT_OTS_3_NR.drop(columns=['OT','Proyecto'],inplace=True)


In [25]:

### Merge a 2 columnas

EA_PAP_PEXT_OTS_2 = pd.merge(EA_PAP_PEXT_OTS_3_NR,OT_agg_ID_PRO,on=['ID_SITIO','PROVEEDOR'],how='left')
EA_PAP_PEXT_OTS_2 = combine_and_rename(EA_PAP_PEXT_OTS_2,[('SITE_x','SITE_y'),('ESTATUS GENERAL','Status OT')],{'SITE_x':'SITE'})

EA_PAP_SAP_PEXT_RLL = pd.concat([EA_PAP_PEXT_OTS_2,EA_PAP_PEXT_OTS_3_R],axis=0)### UNO LOS 2M MERGES

EA_PAP_SAP_PEXT_RLL = EA_PAP_SAP_PEXT_RLL.reset_index(drop=True)

EA_PAP_SAP_PEXT_RLL.Proyecto.replace('STREET CELL/NUEVO RADIOBASE','STREET CELL',inplace=True)
EA_PAP_SAP_PEXT_RLL.Proyecto.replace('NUEVO RADIOBASE','JHORDAN',inplace=True)
EA_PAP_SAP_PEXT_RLL.Proyecto.replace('STREET CELL','DEMETRIO',inplace=True)
EA_PAP_SAP_PEXT_RLL.rename(columns={'RESPONSABLE DE IMPLEMENTACION':'RESPONSABLE_DE_EA'},inplace=True)
EA_PAP_SAP_PEXT_RLL['RESPONSABLE_DE_EA'] = EA_PAP_SAP_PEXT_RLL['RESPONSABLE_DE_EA'].combine_first(EA_PAP_SAP_PEXT_RLL['Proyecto'])
EA_PAP_SAP_PEXT_RLL.drop(columns=['Proyecto'],inplace=True) 
combine_and_rename
EA_PAP_SAP_PEXT_RLL.info() ## final

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1297 entries, 0 to 1296
Data columns (total 60 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   CONCATENADO               1297 non-null   int64         
 1   NUMERO OC                 1297 non-null   int64         
 2   POS_PREC                  1297 non-null   int64         
 3   FECHA_DOC                 1297 non-null   object        
 4   Estrategia de Liberación  1294 non-null   object        
 5   Cod.Solicitante           1297 non-null   object        
 6   SOLICITANTE               1297 non-null   object        
 7   TIPO_DOC                  1297 non-null   object        
 8   TIPO DE MATERIAL          1297 non-null   object        
 9   MATERIAL                  1297 non-null   int64         
 10  TEXTO                     1297 non-null   object        
 11  COD_PROVEEDOR             1294 non-null   float64       
 12  PROVEEDOR           

In [26]:
import pandas as pd

# Merge a 2 columnas
EA_PAP_PEXT_OTS_2 = pd.merge(EA_PAP_PEXT_OTS_3_NR, OT_agg_ID_PRO, on=['ID_SITIO','PROVEEDOR'], how='left')
EA_PAP_PEXT_OTS_2 = combine_and_rename(EA_PAP_PEXT_OTS_2, [('SITE_x','SITE_y'),('ESTATUS GENERAL','Status OT')], {'SITE_x':'SITE'})

# Unir los 2 merges
EA_PAP_SAP_PEXT_RLL = pd.concat([EA_PAP_PEXT_OTS_2, EA_PAP_PEXT_OTS_3_R], axis=0).reset_index(drop=True)

# Reemplazar valores y renombrar columnas
EA_PAP_SAP_PEXT_RLL = (EA_PAP_SAP_PEXT_RLL
                       .replace({'Proyecto': {'STREET CELL/NUEVO RADIOBASE': 'STREET CELL', 
                                              'NUEVO RADIOBASE': 'JHORDAN', 
                                              'STREET CELL': 'DEMETRIO'}})
                       .rename(columns={'RESPONSABLE DE IMPLEMENTACION': 'RESPONSABLE_DE_EA'})
                       .assign(RESPONSABLE_DE_EA=lambda df: df['RESPONSABLE_DE_EA'].combine_first(df['Proyecto'])) ## assign se usa para crear columnas de manera eficiente a partir de ciones lamba
                       .drop(columns=['Proyecto']))

# Información del DataFrame final

## Añado info de Modernizacion 
    - Same primero merge triple y lugo doble en el residuo 

### Merge en 3 columnas

In [27]:
EA_PAP_SAP_PEXT_RLL_M3 = pd.merge(EA_PAP_SAP_PEXT_RLL,OT_cut_M_ID_N_PRO,on=['ID_SITIO','SITE','PROVEEDOR'],how='left')

EA_PAP_SAP_PEXT_RLL_M3 = combine_and_rename(EA_PAP_SAP_PEXT_RLL_M3,[('RESPONSABLE_DE_EA_x','RESPONSABLE_DE_EA_y'),
                                             ('OT_x','OT_y'),('ESTATUS GENERAL','Status OT')],
                                            {'RESPONSABLE_DE_EA_x':'RESPONSABLE_DE_EA','OT_x':'OT',})
EA_PAP_SAP_PEXT_RLL_M_R =EA_PAP_SAP_PEXT_RLL_M3[~EA_PAP_SAP_PEXT_RLL_M3.OT.isna()]
EA_PAP_SAP_PEXT_RLL_M_NA = EA_PAP_SAP_PEXT_RLL_M3[EA_PAP_SAP_PEXT_RLL_M3.OT.isna()].copy()
EA_PAP_SAP_PEXT_RLL_M_NA.drop(columns=['OT'],inplace=True)

### Merge en 2 columnas

In [28]:
EA_PAP_SAP_PEXT_RLL_M2 = pd.merge(EA_PAP_SAP_PEXT_RLL_M_NA,OT_cut_M_ID_PRO,on=['ID_SITIO','PROVEEDOR'],how='left')
EA_PAP_SAP_PEXT_RLL_M2 = combine_and_rename(EA_PAP_SAP_PEXT_RLL_M2,[('RESPONSABLE_DE_EA_x','RESPONSABLE_DE_EA_y'),
                                             ('SITE_x','SITE_y'),('ESTATUS GENERAL','Status OT')],
                                            {'RESPONSABLE_DE_EA_x':'RESPONSABLE_DE_EA','OT_x':'OT','SITE_x':'SITE'})
EA_PAP_SAP_PEXT_RLL_M = pd.concat([EA_PAP_SAP_PEXT_RLL_M_R,EA_PAP_SAP_PEXT_RLL_M2],axis=0)

## Añado info de energia 

In [29]:
EA_PAP_SAP_PEXT_OTS = pd.merge(EA_PAP_SAP_PEXT_RLL_M,OT_cut_E_ID_PRO,on=['ID_SITIO','PROVEEDOR'],how='left')

EA_PAP_SAP_PEXT_OTS = combine_and_rename(EA_PAP_SAP_PEXT_OTS,[('RESPONSABLE_DE_EA_x','RESPONSABLE_DE_EA_y'),
                                                             ('SITE_x','SITE_y'),('ESTATUS GENERAL','Status OT'),
                                                             ('OT_x','OT_y')],
                                                            {'RESPONSABLE_DE_EA_x':'RESPONSABLE_DE_EA','OT_x':'OT','SITE_x':'SITE'})

## Clean Columns

In [30]:
A_PAP_SAP_PEXT_OTS = EA_PAP_SAP_PEXT_OTS[EA_PAP_SAP_PEXT_OTS.ESTATUS == 'EN PROCESO']
EA_PAP_SAP_PEXT_OTS = EA_PAP_SAP_PEXT_OTS.dropna(axis=1, how='all') # elimino columnas vacias
EA_PAP_SAP_PEXT_OTS = EA_PAP_SAP_PEXT_OTS.drop(columns=['F.Creación','REQ','ANTIGUAMIENTO_PAP','NUM_SOT','NATURALEZA','OC Posición'
                                                    ,'T.CAMBIO','AÑO','ESTATUS','PPTO_INICIAL','PPTO_FINAL','RECLA_INTERNA','ID','RECIBIDO_USD'
                                                       ,'MONEDA_ORIG','CONTRATISTA','COMPROMETIDO','RECIBIDO','EN_PROCESO','TIPO_PRES','TIPO_PRY']) # elimino columnas
EA_PAP_SAP_PEXT_OTS.loc[:,'ESTADO_PAP'] = EA_PAP_SAP_PEXT_OTS.loc[:,'ESTADO_PAP'].fillna('Sin Registrar')
EA_PAP_SAP_PEXT_OTS.loc[:,'PROVEEDOR'] = EA_PAP_SAP_PEXT_OTS.loc[:,'PROVEEDOR'].fillna('Sin Proveedor')
EA_PAP_SAP_PEXT_OTS = limpiar_id(EA_PAP_SAP_PEXT_OTS,'ID_SITIO','SITE')
EA_act = EA_PAP_SAP_PEXT_OTS.copy()


In [31]:
cash_out = EA_PAP_SAP_PEXT_OTS['EN_PROC_USD'].sum()
print("La diferencia de montos es: ",Cash_In - cash_out)

La diferencia de montos es:  0.0


In [32]:
EA_PAP_SAP_PEXT_OTS.ESTADO_PAP.value_counts(dropna=False)

ESTADO_PAP
Sin Registrar       934
Pre registro        319
FAC                  11
Visita ejecutada      9
Registrado            6
Programación ATP      4
Aprobaciones FAC      4
Observado             3
PAC                   3
Rechazado             2
Programado            2
Name: count, dtype: int64

## 2 Excel

In [33]:
if reporte_old == False:
    df_analist,_ = get_recent_df(r'C:\Users\C26764\America Movil Peru S.A.C\EAS - 1',sheet_name='Sheet1')
    df_analist.rename(columns={'RESPONSABLE DE EA':'RESPONSABLE_DE_EA'},inplace=True)
    df_analist_cut = df_analist[['CONCATENADO','SITE','Estado de EA','RESPONSABLE_DE_EA','ESTATUS GENERAL']].copy()

    EA_ACT_A_D = pd.merge(EA_act,df_analist_cut,on=['CONCATENADO'],how='left')
    EA_ACT_A_D = combine_and_rename(EA_ACT_A_D,[('SITE_x', 'SITE_y')],{'SITE_x':'SITE'})

    EA_ACT_A_D = update_and_rename(EA_ACT_A_D,[('RESPONSABLE_DE_EA_x', 'RESPONSABLE_DE_EA_y'),
                                              ('ESTATUS GENERAL_x','ESTATUS GENERAL_y')],
                                   {'RESPONSABLE_DE_EA_x':'RESPONSABLE_DE_EA',
                                    'ESTATUS GENERAL_x':'ESTATUS GENERAL'})

    
    useless_columns= ['PEP','TIPO DE MATERIAL','SUB_DIRECCION','Estrategia de Liberación','MATERIAL',
                          'Cod.Solicitante','TIPO DE MATERIAL','TIPO_DOC','TIPO_PROYECTOS','COD_PROVEEDOR',
                      'POS_PRESUP','COMPROM_USD','CE_GESTOR','RUBRO','CENTRO_GESTOR','Proyecto CAPEX','Cod.Solicitante'
                      ,'Proyecto CAPEX','AÑO','PPTO_INICIAL','RECLA_INTERNA','PPTO_FINAL','Estrategia de Liberación'
                     'MONEDA_ORIG','POS_PRESUP']
    
    EA_ACT_A_D['Estado de EA'].fillna('PENDIENTE',inplace=True)
    print("se crea nuevo archivo editable")
    Ac.Excel_format(EA_ACT_A_D,fr'C:\Users\C26764\America Movil Peru S.A.C\EAS - 1\EAUPDATE{Today_D_M}.xlsx',useless_columns) 
    
else: 
    print("no se crea aditable")
    pass

no se crea aditable


### Cargo lista de OCS que ya se asignaron fecha de Ejecucion

In [34]:
with open(r'D:\Scripts1\Code\ActPEA\CODE\Temps\DF_OCS.pkl', "rb") as archivo:
    Ocs_DF = pickle.load(archivo)
Ocs_DF = Ocs_DF.drop_duplicates()
Ocs_F = Ocs_DF['CONCATENADO'].tolist() 

df_analist,_ = get_recent_df(r'C:\Users\C26764\America Movil Peru S.A.C\EAS - 1',sheet_name='Sheet1')
df_yesterday,_ = get_recent_csv(r'D:\EA\Resultados')
##Merge 
df_analist = pd.merge(df_analist,df_yesterday[['CONCATENADO','Fecha de Ejecucion']],on='CONCATENADO',how='left')
#df_analist.loc[:,'Fecha de Ejecucion'] = df_yesterday.loc[:,'Fecha de Ejecucion']
#df_analist.loc[:,'Fecha de Ejecucion'] = pd.NaT
df_analist_cut = df_analist[['CONCATENADO','SITE','Estado de EA','RESPONSABLE_DE_EA','ESTATUS GENERAL','Fecha de Ejecucion']].copy()
df_analist_cut.loc[:,'ESTATUS GENERAL'].replace('TERMINADO','EJECUTADO',inplace=True)

df_row_diff = df_analist_cut[df_analist_cut['ESTATUS GENERAL'].ne(df_yesterday['ESTATUS GENERAL'])]
df_row_diff = df_row_diff[df_row_diff['ESTATUS GENERAL'] == 'EJECUTADO']
df_row_diff = df_row_diff[~df_row_diff['CONCATENADO'].isin(Ocs_F)]


import datetime
df_row_diff['Fecha de Ejecucion'] = datetime.date.today().strftime('%d-%m-%Y')
df_row_diff['Fecha de Ejecucion'] = pd.to_datetime(df_row_diff['Fecha de Ejecucion'],format = '%d-%m-%Y')


C:\Users\C26764\America Movil Peru S.A.C\EAS - 1\EAUPDATE01.07.xlsx
D:\EA\Resultados\EAS_03-07-2024.csv


In [35]:
df_analist['Estado de EA'].value_counts(dropna=False)

Ocs_DF = pd.concat([Ocs_DF,df_row_diff[['CONCATENADO','Fecha de Ejecucion']]],axis=0)

with open(r'D:\Scripts1\Code\ActPEA\CODE\Temps\DF_OCS.pkl', "wb") as archivo:
    pickle.dump(Ocs_DF, archivo)

In [36]:
df_row_diff

,CONCATENADO,SITE,Estado de EA,RESPONSABLE_DE_EA,ESTATUS GENERAL,Fecha de Ejecucion
569,450068479610,LA_UNION_II,PENDIENTE,ANGGIE,EJECUTADO,2024-07-04
771,450071301040,MANCORA_NORTE,PENDIENTE,Danner Yarleque,EJECUTADO,2024-07-04


In [37]:
df_analist_cut.update(df_row_diff)


## Merge data of the analist

In [38]:
analistas_map = {'JENNY PIZAN':'JENNY',
                 'DANNER YARLEQUE':'DANNER',
                 'LAURA RAFAEL':'LAURA'}
Estado_EA_map = {'POR LIQUIDAR':'PENDIENTE'}

In [39]:
df_analist_cut['Fecha de Ejecucion'] = pd.to_datetime(df_analist_cut['Fecha de Ejecucion'], unit='ns')
EA_ACT_A_D = pd.merge(EA_PAP_SAP_PEXT_OTS,df_analist_cut,on=['CONCATENADO'],how='left')
EA_ACT_A_D = combine_and_rename(EA_ACT_A_D,[('SITE_x', 'SITE_y')],{'SITE_x':'SITE'})
EA_ACT_A_D = update_and_rename(EA_ACT_A_D,[('RESPONSABLE_DE_EA_x', 'RESPONSABLE_DE_EA_y'),
                                          ('ESTATUS GENERAL_x','ESTATUS GENERAL_y')],
                               {'RESPONSABLE_DE_EA_x':'RESPONSABLE_DE_EA','ESTATUS GENERAL_x':'ESTATUS GENERAL'})

#EA_ACT_A_D['Fecha de Ejecucion'] = pd.to_datetime(EA_ACT_A_D['Fecha de Ejecucion'], errors='coerce')

#display(EA_ACT_A_D[~EA_ACT_A_D['Fecha de Ejecucion'].isna()])
EA_ACT_A_D.loc[:,'RESPONSABLE_DE_EA'] = EA_ACT_A_D.loc[:,'RESPONSABLE_DE_EA'].fillna('Por asignar') 
EA_ACT_A_D.loc[:,'Estado de EA'] = EA_ACT_A_D.loc[:,'Estado de EA'].fillna('PENDIENTE') 

# Estos son los valores permitidos
valores_permitidos = ['PENDIENTE', 'EJECUTADO', 'ANULAR', 'ANULADA','LIQUIDADO']

valores_anuladas = ['OC ANULADA']

valores_anular = ['PENDIENTE ANULAR']


# Este es el valor que se asignará cuando el valor actual no esté en la lista de valores permitidos
valor_asignado = 'PENDIENTE'

EA_ACT_A_D['Estado de EA'] = EA_ACT_A_D['Estado de EA'].apply(lambda x: 'ANULADA' if x in valores_anuladas else x)
EA_ACT_A_D['Estado de EA'] = EA_ACT_A_D['Estado de EA'].apply(lambda x: 'ANULAR' if x in valores_anular else x)

# Aplica la condición a la columna
EA_ACT_A_D['Estado de EA'] = EA_ACT_A_D['Estado de EA'].where(EA_ACT_A_D['Estado de EA'].isin(valores_permitidos), valor_asignado)


EA_ACT_A_D.loc[:,'RESPONSABLE_DE_EA'] = EA_ACT_A_D.loc[:,'RESPONSABLE_DE_EA'].str.upper()
EA_ACT_A_D.loc[:,'RESPONSABLE_DE_EA'] = EA_ACT_A_D.loc[:,'RESPONSABLE_DE_EA'].replace(analistas_map)

EA_ACT_A_D.loc[:,'Estado de EA'].replace('EJECUTADA','EJECUTADO',inplace = True)
### Creo columna de mes para agrupar 
EA_ACT_A_D['FECHA_DOC'] = EA_ACT_A_D['FECHA_DOC'].str.replace(' 00:00:00', '')
EA_ACT_A_D['FECHA_DOC'] = pd.to_datetime(EA_ACT_A_D['FECHA_DOC'],format = '%Y-%m-%d')




In [40]:
EA_ACT_A_D['ESTATUS GENERAL'].value_counts(dropna=False)

ESTATUS GENERAL
NaN             472
CW              191
POR EJECUTAR    149
EJECUTADO       105
TI               77
EN EJECUCION     70
ON AIR           49
CW/TI            46
OH               40
ACTIVO           32
ING              24
SAQ              16
SAQ/ING           6
NO INICIADO       6
ING/ON AIR        3
ING/RF            3
ING/SAQ           3
ANULAR            2
FINALIZADO        1
CW/TI/ING         1
CANCELADO         1
Name: count, dtype: int64

In [41]:
EA_ACT_A_D['Estado de EA'].value_counts(dropna=False)

Estado de EA
PENDIENTE    1280
ANULAR         12
EJECUTADO       4
LIQUIDADO       1
Name: count, dtype: int64

In [42]:
EA_ACT_A_D.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1297 entries, 0 to 1296
Data columns (total 38 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   CONCATENADO               1297 non-null   int64         
 1   NUMERO OC                 1297 non-null   int64         
 2   POS_PREC                  1297 non-null   int64         
 3   FECHA_DOC                 1297 non-null   datetime64[ns]
 4   Estrategia de Liberación  1294 non-null   object        
 5   Cod.Solicitante           1297 non-null   object        
 6   SOLICITANTE               1297 non-null   object        
 7   TIPO_DOC                  1297 non-null   object        
 8   TIPO DE MATERIAL          1297 non-null   object        
 9   MATERIAL                  1297 non-null   int64         
 10  TEXTO                     1297 non-null   object        
 11  COD_PROVEEDOR             1294 non-null   float64       
 12  PROVEEDOR           

In [43]:
EA_ACT_A_D['RESPONSABLE_DE_EA'].replace('ANNGIE','ANGGIE',inplace=True)

In [44]:
import datetime
## guardo el TSLM
filename = r'D:\EA\TS\last_run.json'
# Carga la última fecha de ejecución
last_run_date = Ac.load_last_run_date(filename)

# Comprueba si la celda ya se ha ejecutado hoy
if last_run_date != datetime.datetime.now().date():
    # Tu código aquí
    print('Ejecutado: ',Today_str)
    #process_to_bcsv(PRE_all_act,'D:/Prepa/TIME.S/Prepa_TS1.csv',Today_str)
    EA_ACT_A_D.to_csv(fr'D:\EA\Resultados\EAS_{Today_str}.csv',index=False) ## guardo el TODAY
    process_to_bcsv(EA_ACT_A_D.copy(),r'D:\EA\TS\EA_TS1.csv',Today_str)# añado al acumulado

    # Guarda la fecha de hoy como la última fecha de ejecución
    Ac.save_last_run_date(filename)
else:
    print("El código ya se ha ejecutado hoy.")

Ejecutado:  04-07-2024


# TO BI 
    -Proceso el DF consolidado para el BI 

In [45]:
EA_ACT_A_D.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1297 entries, 0 to 1296
Data columns (total 38 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   CONCATENADO               1297 non-null   int64         
 1   NUMERO OC                 1297 non-null   int64         
 2   POS_PREC                  1297 non-null   int64         
 3   FECHA_DOC                 1297 non-null   datetime64[ns]
 4   Estrategia de Liberación  1294 non-null   object        
 5   Cod.Solicitante           1297 non-null   object        
 6   SOLICITANTE               1297 non-null   object        
 7   TIPO_DOC                  1297 non-null   object        
 8   TIPO DE MATERIAL          1297 non-null   object        
 9   MATERIAL                  1297 non-null   int64         
 10  TEXTO                     1297 non-null   object        
 11  COD_PROVEEDOR             1294 non-null   float64       
 12  PROVEEDOR           

In [63]:
Columnas_2_use= EA_ACT_A_D[['NUMERO OC','POS_PREC','FECHA_DOC','SOLICITANTE','TEXTO','PROVEEDOR',
                 'EN_PROC_USD','CE_GESTOR','TIPO_PROYECTOS'
                           ]]
last_columns = EA_ACT_A_D.iloc[:,-12:]
EA_ACT_A_B = pd.concat([Columnas_2_use,last_columns], axis=1)

In [64]:
list_columns = ['TEXTO' ,'SITE','ESTATUS GENERAL','TIPO_PROYECTOS']
for column in list_columns:
    EA_ACT_A_B[column] = EA_ACT_A_B[column].str.capitalize()

In [65]:
## capitalizar valaores de la columna que esatn todas en mayuculas

In [66]:
EA_ACT_A_B.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1297 entries, 0 to 1296
Data columns (total 21 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   NUMERO OC           1297 non-null   int64         
 1   POS_PREC            1297 non-null   int64         
 2   FECHA_DOC           1297 non-null   datetime64[ns]
 3   SOLICITANTE         1297 non-null   object        
 4   TEXTO               1297 non-null   object        
 5   PROVEEDOR           1297 non-null   object        
 6   EN_PROC_USD         1297 non-null   float64       
 7   CE_GESTOR           1297 non-null   object        
 8   TIPO_PROYECTOS      1297 non-null   object        
 9   CLASIF_FINANZAS     1297 non-null   object        
 10  CLASIF_RED_1        1297 non-null   object        
 11  CLASIF_RED_2        1297 non-null   object        
 12  ID_SITIO            1297 non-null   object        
 13  SITE                1297 non-null   object      

In [67]:
EA_ACT_A_B['Estado de EA'].value_counts()

Estado de EA
PENDIENTE    1280
ANULAR         12
EJECUTADO       4
LIQUIDADO       1
Name: count, dtype: int64

In [68]:
EA_ACT_A_B.to_csv(r'\\LIMBIPBICOV01.claro.pe\Red Región Norte\EAS\EAs.csv',index=False) 